# Topic Modelling Presidential Inaugural Addresses

In [44]:
%matplotlib inline

import os
import re
import pprint
import pandas as pd
import nltk
import gensim
from textblob import TextBlob, Word
from nltk.corpus import stopwords, inaugural
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_colwidth = 0
pp = pprint.PrettyPrinter(indent=4)

# Data
The dataset we will use for topic modelling will be the presidential inaugural addresses, from George Washington to Barack Obama (2009)

The first thing we will do is process the raw speeches, and then putting them into a dataframe (think a database table). The dataframe will contain metadata about the speeches - year, historical era, and the president's name.

Our `process` function does a few key things:
* Filters out **stopwords**, which typically refers to the most common words in a language. e.g. "the", "is", "at", ...
* Filters out punctuation and numbers.
* Makes sure that no empty strings are present.
* makes every word lowercase.

In [281]:
# this will allow us to map inaugural years to historical eras.
def historical_era(year):
    if year < 1830:
        return "Early Republic"
    elif year < 1854:
        return "Jacksonian Democracy"
    elif year < 1882:
        return "Sectional Conflict"
    elif year < 1898:
        return "Gilded Age"
    elif year < 1923:
        return "Progressive Era"
    elif year < 1962:
        return "Depression and World Conflict"
    elif year < 1990:
        return "Social Change and Soviet Relations"
    else:
        return "Globalization"
    
def process(speech):
    stoplist = set(stopwords.words())
    return [re.sub(r'--|;|:|\(|\.|\,|\)|[0-9]*',"", word) for word in speech.lower().split() if word not in stoplist]

In [282]:
raw_speeches = [inaugural.raw(fileid) for fileid in inaugural.fileids()]
years = [int(fileid[:4]) for fileid in inaugural.fileids()]
presidents = [fileid.split("-")[1].replace(".txt","") for fileid in inaugural.fileids()]

# strip out stopwords 
speeches = [process(speech) for speech in raw_speeches]
speeches = [[word for word in speech if word !=""]for speech in speeches]

In [285]:
speeches_df = pd.DataFrame(zip(raw_speeches,speeches, years, presidents), columns = ["raw_speech","speech", "year", "president"])
speeches_df["era"] = speeches_df["year"].apply(historical_era) 

# we will use this later when analyzing how topics change over the course of time.
eras = list(set(list(speeches_df["era"])))

Here, we see what our dataframe looks like. 

| raw_speech | speech | year | president | era |
| --- | --- | --- | --- | --- | --- |
| exact speech | processed text | year of speech | speaker | historical era |


In [286]:
speeches_df.head(1)

raw_speech  \
0  Fellow-Citizens of the Senate and of the House of Representatives:\n\nAmong the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the 14th day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years -- a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my country called me, being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications, could not but overwhelm with despondence one who (inheriting inferior endowments from nature and unpracticed in the duties of civil administration) ought to be peculiarly conscious of his own deficiencies. In this conflict of emotions all I dare aver is that it has been my faithful study to collect my duty from a just appreciation of every circumstance by which it might be affected. All I dare hope is that if, in executing this task, I have been too much swayed by a grateful remembrance of former instances, or by an affectionate sensibility to this transcendent proof of the confidence of my fellow citizens, and have thence too little consulted my incapacity as well as disinclination for the weighty and untried cares before me, my error will be palliated by the motives which mislead me, and its consequences be judged by my country with some share of the partiality in which they originated.\n\nSuch being the impressions under which I have, in obedience to the public summons, repaired to the present station, it would be peculiarly improper to omit in this first official act my fervent supplications to that Almighty Being who rules over the universe, who presides in the councils of nations, and whose providential aids can supply every human defect, that His benediction may consecrate to the liberties and happiness of the people of the United States a Government instituted by themselves for these essential purposes, and may enable every instrument employed in its administration to execute with success the functions allotted to his charge. In tendering this homage to the Great Author of every public and private good, I assure myself that it expresses your sentiments not less than my own, nor those of my fellow citizens at large less than either. No people can be bound to acknowledge and adore the Invisible Hand which conducts the affairs of men more than those of the United States. Every step by which they have advanced to the character of an independent nation seems to have been distinguished by some token of providential agency; and in the important revolution just accomplished in the system of their united government the tranquil deliberations and voluntary consent of so many distinct communities from which the event has resulted can not be compared with the means by which most governments have been established without some return of pious gratitude, along with an humble anticipation of the future blessings which the past seem to presage. These reflections, arising out of the present crisis, have forced themselves too strongly on my mind to be suppressed. You will join with me, I trust, in thinking that there are none under the influence of which the proceedings of a new and free government can more auspiciously commence.\n\nBy the article establishing the executive department it is made the duty of the President "to recommend to your consideration such measures as he shall judge necessary and expedient." The circumstances under which I now m

## Transformations

Next, we will transform all of our speeches using a document representation called **bag-of-words**. 

In this representation, each document is represented by one vector where each vector element represents a question-answer pair, in the style of:

>“How many times does the word *protest* appear in the document? Once.”

It is advantageous to represent the questions only by their (integer) ids. The mapping between the questions and ids is called a dictionary.

We see that after filtering out stopwords, and in/frequent words, we are left with a vocabulary consisting on 3217 unique tokens.

In [390]:
# create a dictionary that maps words to integers.
dictionary = gensim.corpora.Dictionary(speeches)

# filter out really frequent and infrequent words
dictionary.filter_extremes(no_below=2)

dictionary.save("../data/reviews.dict")
print(dictionary)

Dictionary(4934 unique tokens: [u'aided', u'limited', u'dissolution', u'comparatively', u'desirable']...)


>What are the 3 most common words in Washington's first inaugural address? How about Obama's 2009 address?

In [391]:
print "Washington, 1789: ", pp.pformat([(dictionary[word[0]], word[1]) for word in sorted(dictionary.doc2bow(speeches[0]), key=lambda x: x[1], reverse=True)[:3]])
print "Obama, 2009: ", pp.pformat([(dictionary[word[0]], word[1]) for word in sorted(dictionary.doc2bow(speeches[-1]), key=lambda x: x[1], reverse=True)[:3]])

Washington, 1789:  [(u'me', 5), (u'ought', 4), (u'nature', 3)]
Obama, 2009:  [(u'america', 7), (u'today', 6), (u'cannot', 6)]


Once we have our dictionary, we can now convert each document into it's **bag-of-words** representation and save the collection of these vectors as our corpus. Each entry in our corpus will indicate which words from our dictioary appear in the address, and how often.

In [392]:
corpus = [dictionary.doc2bow(speech) for speech in speeches]
gensim.corpora.MmCorpus.serialize("../data/reviews.mm", corpus)
print "Obama, 2009"
print "Bag-of-Words: ", pp.pformat(corpus[-1])
print "Processed: ", pp.pformat([dictionary[id] for id,_ in corpus[-1]])

Obama, 2009
Bag-of-Words:  [   (32, 3),
    (37, 1),
    (41, 1),
    (44, 1),
    (57, 1),
    (71, 1),
    (72, 2),
    (76, 1),
    (82, 3),
    (112, 1),
    (119, 1),
    (121, 2),
    (125, 1),
    (133, 1),
    (139, 1),
    (173, 1),
    (179, 1),
    (182, 1),
    (184, 1),
    (190, 1),
    (201, 1),
    (209, 1),
    (215, 1),
    (218, 1),
    (222, 1),
    (233, 1),
    (238, 1),
    (242, 2),
    (253, 1),
    (254, 2),
    (258, 1),
    (267, 1),
    (272, 1),
    (274, 1),
    (275, 1),
    (301, 1),
    (308, 1),
    (345, 1),
    (349, 1),
    (350, 1),
    (354, 2),
    (364, 2),
    (372, 1),
    (380, 1),
    (384, 1),
    (392, 1),
    (406, 1),
    (409, 1),
    (411, 1),
    (412, 2),
    (415, 1),
    (425, 1),
    (475, 1),
    (481, 1),
    (485, 1),
    (514, 1),
    (520, 1),
    (528, 1),
    (529, 2),
    (535, 1),
    (543, 2),
    (550, 1),
    (555, 1),
    (563, 1),
    (582, 1),
    (583, 1),
    (584, 1),
    (585, 1),
    (624, 1),
    (632, 1),
  

# Topic Modelling

We are now equipped to perform topic modelling on the speeches. However, our current corpus treats all words equally. That is, all terms carry equal weight across all of the speeches. Another transformation called **td-idf** (term frequency - inverse document frequency) will update the weights of each word so as to capture how important it is to the speech. 

In [393]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
# corpus_tfidf = tfidf[corpus]

Using our td-idf representation, we will perform **Latent Dirichlet Allocation (LDA)**, which will allow us to discover topics in the inaugural speeches. 

The model outputs a series of "topics" which are comprised of words and weights corresponding to the "influence" that word has on the topic. 

In [394]:
lda = gensim.models.ldamodel.LdaModel(corpus, id2word = dictionary, num_topics = 30, passes=4)
corpus_lda = lda[corpus_tfidf]

# print 10 of our 30 topics.
pp.pprint(lda.print_topics(10, num_words=3))

[   (9, u'0.001*"executive" + 0.001*"congress" + 0.001*"subject"'),
    (25, u'0.006*"clothed" + 0.004*"housed" + 0.004*"fed"'),
    (7, u'0.009*"sides" + 0.008*"pledge" + 0.006*"ask"'),
    (11, u'0.012*"democracy" + 0.006*"america" + 0.006*"will"'),
    (17, u'0.002*"america" + 0.002*"congress" + 0.001*"executive"'),
    (24, u'0.006*"address" + 0.005*"neither" + 0.005*"slaves"'),
    (18, u'0.001*"executive" + 0.001*"congress" + 0.001*"general"'),
    (21, u'0.004*"congress" + 0.003*"republic" + 0.003*"general"'),
    (16, u'0.007*"america" + 0.006*"today" + 0.004*"things"'),
    (26, u'0.014*"america" + 0.014*"responsibility" + 0.010*"abroad"')]


This allows us to inspect the topic that each of the documents belongs to

In [410]:
check = 1
print speeches_df.iloc[check]["president"], ",", speeches_df.iloc[check]["year"]
print sorted(lda.get_document_topics(corpus[check]), key = lambda x: x[1], reverse=True)
print pp.pformat(lda.print_topic(22))

Washington , 1793
[(4, 0.97156862745096717)]
u'0.006*"false" + 0.006*"reason" + 0.005*"limits" + 0.005*"due" + 0.005*"press" + 0.005*"therefore" + 0.004*"measures" + 0.004*"truth" + 0.004*"expenses" + 0.004*"revenue"'


In [412]:
print [dictionary[word[0]] for word in lda.get_topic_terms(4)]
print speeches_df.iloc[0]["raw_speech"]

[u'congress', u'executive', u'action', u'revenue', u'institutions', u'business', u'nothing', u'countrymen', u'party', u'citizen']
Fellow-Citizens of the Senate and of the House of Representatives:

Among the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the 14th day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years -- a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my country called me, being su

In [402]:
# check how many topics each speech belongs to
for i, _ in enumerate(corpus):
    print i, ": ", len(lda.get_document_topics(corpus[i]))

0 :  1
1 :  1
2 :  1
3 :  1
4 :  1
5 :  2
6 :  1
7 :  3
8 :  3
9 :  3
10 :  1
11 :  2
12 :  5
13 :  4
14 :  3
15 :  1
16 :  3
17 :  3
18 :  5
19 :  1
20 :  1
21 :  2
22 :  2
23 :  3
24 :  2
25 :  6
26 :  2
27 :  2
28 :  2
29 :  1
30 :  1
31 :  1
32 :  1
33 :  6
34 :  5
35 :  4
36 :  4
37 :  1
38 :  3
39 :  1
40 :  1
41 :  3
42 :  3
43 :  1
44 :  2
45 :  1
46 :  3
47 :  1
48 :  2
49 :  1
50 :  2
51 :  1
52 :  2
53 :  2
54 :  3
55 :  1


# INTRODUCE NEW SENTENCES. ALSO, DOCUMENT SIMILARITY.

# Exercises

1. What happens if you split up the data by era and run an LDA on each of the eras separately? How do the topics change over time.
2. Label the topics!
3. Play around with parameters, how do the topics change?
  * try filtering the dictionary differently.
  * try changing the number of topics that you are hardcoding.
4. How can you change the processing step to add more intelligent parsing of the speeches?

## Appendix

Gensim Link
https://algobeans.com/2015/06/21/laymans-explanation-of-topic-modeling-with-lda-2/
http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/